<img src="assets/projet7_logo.svg" alt="Projet 7" style="float: right; width: 120px; margin: 0 0 0 12px;" />

# Projet 7 
**Auteur : El-yassa Zebakh**

# Home Credit Scoring — Notebook de modélisation

Notebook rédigé pour documenter le pipeline de scoring construit sur le dataset Home Credit.


**Ce que fait ce notebook**
- Charge le dataset `application_train.csv` et dresse un diagnostic rapide (types, valeurs manquantes, distribution de la cible TARGET).
- Met en place un pipeline de prétraitement scikit-learn (numérique + catégoriel) en évitant toute fuite de données.
- Entraîne plusieurs modèles (régression logistique, gradient boosting, XGBoost/LightGBM) via RandomizedSearchCV.
- Optimise un seuil métier basé sur un coût FN/FP asymétrique, sélectionne un modèle champion et logue les résultats dans MLflow.


### Sommaire
- [1. Import des dépendances et chargement initial des données](#1-import-des-dependances-et-chargement-initial-des-donnees)
- [2. Typologie des variables et valeurs manquantes](#2-typologie-des-variables-et-valeurs-manquantes)
- [3. Cible métier (TARGET)](#3-cible-metier-target)
- [4. Prétraitement des données](#4-pretraitement-des-donnees-pipelines-scikit-learn)
- [4.1 Règles générales et split train/validation/test](#41-regles-generales-et-split-trainvalidationtest)
- [4.2 Séparer numériques / catégorielles](#42-separer-numeriques-categorielles)
- [4.3 Pipeline numérique](#43-pipeline-numerique)
- [4.4 Pipeline catégoriel](#44-pipeline-categoriel)
- [4.5 ColumnTransformer et pipeline global](#45-columntransformer-et-pipeline-global)
- [5. Modèles candidats et recherche d'hyperparamètres](#5-modeles-candidats-et-recherche-dhyperparametres)
- [6. Évaluation validation : AUC et coût métier](#6-evaluation-validation-auc-et-cout-metier)
- [7. Sélection du modèle champion et interprétation métier](#7-selection-du-modele-champion-et-interpretation-metier)
- [8. Suivi expérimental avec MLflow](#8-suivi-experimental-avec-mlflow)


## <a id="1-import-des-dependances-et-chargement-initial-des-donnees"></a>1. Import des dépendances et chargement initial des données


In [ ]:
import pandas as pd 
import missingno as msno
import sys
from pathlib import Path

# On suppose que le notebook est dans HOME_CREDIT_PROJECT/notebooks
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent
MLFLOW_DIR = PROJECT_ROOT / "mlruns"

# On ajoute la racine du projet au sys.path 
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

print("PROJECT_ROOT =", PROJECT_ROOT)
print("src in sys.path ?", any("home_credit_project" in p and "src" in p for p in sys.path))

In [ ]:
df = pd.read_csv('../data/raw/application_train.csv')
df.head()
df.info()
df.describe()
df.isnull().sum()
msno.bar(df)


## <a id="2-typologie-des-variables-et-valeurs-manquantes"></a>2. Typologie des variables et valeurs manquantes


In [ ]:
categorical_df = df.select_dtypes(include=['object', 'category', 'bool'])
numerical_df = df.select_dtypes(include=['number'])

print(f"Number of categorical features: {categorical_df.shape[1]}")
print(f"Number of numerical features: {numerical_df.shape[1]}")

missing_summary = (
    df.isna()
      .mean()
      .mul(100)
      .rename('missing_pct')
      .round(2)
      .to_frame()
)

categorical_cardinality = (
    categorical_df.apply(lambda s: s.nunique(dropna=True))
      .rename('unique_categories')
      .sort_values(ascending=False)
      .to_frame()
)

with pd.option_context('display.max_rows', 122):
    display(missing_summary[missing_summary['missing_pct'] > 0].sort_values(by='missing_pct', ascending=False))
display(categorical_cardinality)


## <a id="3-cible-metier-target"></a>3. Cible métier (TARGET)


In [ ]:

TARGET_COL = 'TARGET'

value_counts = df[TARGET_COL].value_counts(dropna=False)
percentages = df[TARGET_COL].value_counts(normalize=True, dropna=False).mul(100).round(2)

target_summary = (
    pd.DataFrame({'count': value_counts, 'percent': percentages})
      .sort_index()
)

print(f"Value counts for target '{TARGET_COL}':")
display(target_summary)


## <a id="4-pretraitement-des-donnees-pipelines-scikit-learn"></a>4. Prétraitement des données (pipelines scikit-learn)


Prétraitement des variables (pipeline scikit-learn)

### <a id="41-regles-generales-et-split-trainvalidationtest"></a>4.1 Règles générales et split train/validation/test


In [ ]:
"""
Afin de garantir une démarche reproductible et d’éviter toute fuite de données (data leakage), 
l’ensemble des étapes de prétraitement a été encapsulé dans un pipeline scikit-learn. Ce pipeline est systématiquement appris sur le jeu d’entraînement uniquement, 
puis appliqué aux jeux de validation, de test et, à terme, aux nouvelles demandes de crédit.

"""

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]

# 1) Train/Test split avec stratification
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y,
)

# 2) Train/Validation split basé sur la portion train_val (aussi stratifié)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_val, y_train_val,
    test_size=0.2,  
    random_state=42,
    stratify=y_train_val,
)

print('Shapes:')
print(' X_train:', X_train.shape, ' y_train:', y_train.shape)
print(' X_valid:', X_valid.shape, ' y_valid:', y_valid.shape)
print(' X_test :', X_test.shape,  ' y_test :', y_test.shape)

print('\nClass balance (%):')
for name, yy in [('train', y_train), ('valid', y_valid), ('test', y_test)]:
    pct = (yy.value_counts(normalize=True) * 100).round(2)
    print(f' {name}:')
    print(pct.sort_index())


### <a id="42-separer-numeriques-categorielles"></a>4.2 Séparer numériques / catégorielles


Séparation numériques / catégorielles

In [ ]:

"""
Les variables explicatives ont été séparées en deux groupes :
	•	variables numériques (montants, durées, scores, ratios…),
	•	variables catégorielles (type de contrat, type de logement, situation familiale, etc.).

Les colonnes techniques, telles que l’identifiant de dossier SK_ID_CURR, ainsi que les variables présentant plus de 60 % de valeurs manquantes,
 ont été supprimées sur la base du jeu d’entraînement, puis ce même choix de variables a été appliqué aux jeux de validation et de test.
 
 """

In [ ]:

MISSING_THRESHOLD = 0.60

missing_ratio_train = X_train.isna().mean()
cols_to_drop = missing_ratio_train[missing_ratio_train > MISSING_THRESHOLD].index.tolist()


if 'SK_ID_CURR' in X_train.columns and 'SK_ID_CURR' not in cols_to_drop:
    cols_to_drop.append('SK_ID_CURR')

print(f"Columns to drop (> {int(MISSING_THRESHOLD*100)}% missing + SK_ID_CURR): {len(cols_to_drop)}")

X_train = X_train.drop(columns=cols_to_drop)
X_valid = X_valid.drop(columns=cols_to_drop)
X_test  = X_test.drop(columns=cols_to_drop)

feature_df = X_train  

numeric_features = feature_df.select_dtypes(include=['number']).columns.tolist()
categorical_features = feature_df.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()

print(f"Numeric features: {len(numeric_features)} | Categorical features: {len(categorical_features)}")


### <a id="43-pipeline-numerique"></a>4.3 Pipeline numérique


Prétraitement des variables numériques

In [ ]:
""" 
	1.	Imputation des valeurs manquantes par la médiane :
La médiane est calculée sur le jeu d’entraînement uniquement, ce qui la rend robuste aux valeurs extrêmes fréquemment observées dans les données financières 
(revenus très élevés, montants de crédit atypiques).

	2.	Standardisation (centrage-réduction) :
Chaque variable est centrée et réduite (moyenne 0, écart-type 1) à partir des statistiques du jeu d’entraînement. 
Cette étape est particulièrement importante pour les modèles linéaires tels que la régression logistique, afin d’éviter que certaines variables dominent numériquement 
les autres et de faciliter la convergence de l’algorithme d’optimisation.”

"""

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

numeric_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
    ]
)




### <a id="44-pipeline-categoriel"></a>4.4 Pipeline catégoriel


Prétraitement des variables catégorielles

In [ ]:
""" 
	1.	Imputation des valeurs manquantes :
Les valeurs manquantes sont remplacées par une modalité dédiée ‘MISSING’, calculée à partir du jeu d’entraînement.

	2.	Encodage One-Hot :
Les variables catégorielles sont ensuite transformées en indicatrices binaires via un encodage One-Hot. Afin d’assurer la robustesse du modèle en production, 
les catégories jamais observées lors de l’entraînement sont explicitement gérées (elles sont ignorées lors de l’inférence, ce qui évite les erreurs).”

"""

In [ ]:
from sklearn.preprocessing import OneHotEncoder

categorical_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='MISSING')),
        ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ]
)

### <a id="45-columntransformer-et-pipeline-global"></a>4.5 ColumnTransformer et pipeline global


ColumnTransformer et pipeline global

In [ ]:
"""
Les deux pipelines (numérique et catégoriel) sont combinés à l’aide d’un ColumnTransformer, qui applique en parallèle le traitement adapté à chaque groupe de variables, 
puis concatène le tout pour obtenir une matrice de caractéristiques prête à être exploitée par les modèles.

Ce préprocesseur global est ensuite intégré comme première étape dans l’ensemble des pipelines de modèles (régression logistique, Gradient Boosting, XGBoost/LightGBM). 
Ainsi, lors de la cross-validation et de l’optimisation des hyperparamètres (RandomizedSearchCV), ce sont toujours les données brutes qui sont fournies au pipeline, 
et le prétraitement est réappris à chaque fold à partir du sous-jeu d’entraînement du fold. Cela garantit une évaluation honnête des performances et une absence de fuite de données.

"""

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', numeric_transformer, numeric_features),
        ('categorical', categorical_transformer, categorical_features)
    ],
    remainder='drop'
)

preprocessor.set_output(transform='pandas')

X_train_prepared = preprocessor.fit_transform(X_train)
X_valid_prepared = preprocessor.transform(X_valid)
X_test_prepared = preprocessor.transform(X_test)

X_train_prepared.head()


## <a id="5-modeles-candidats-et-recherche-dhyperparametres"></a>5. Modèles candidats et recherche d'hyperparamètres


In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.utils.class_weight import compute_class_weight
from xgboost import XGBClassifier

preprocessor = globals().get('preprocessor')


y_train_values = y_train.to_numpy()
classes = np.unique(y_train_values)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train_values)
class_weight_dict = dict(zip(classes, class_weights))

# scale_pos_weight pour XGBoost 
if len(classes) == 2:
    counts = y_train.value_counts()
    # positive class assumed to be the larger label value
    majority_class = counts.idxmax()
    minority_class = counts.idxmin()
    scale_pos_weight = counts[majority_class] / counts[minority_class]
else:
    scale_pos_weight = 1.0

pipeline_lr = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression(
            class_weight=class_weight_dict,
            max_iter=3000,
            solver='lbfgs'
        )),
    ]
)

pipeline_gb = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('classifier', HistGradientBoostingClassifier(
            class_weight=class_weight_dict,
            random_state=42
        )),
    ]
)

pipeline_xgb = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('classifier', XGBClassifier(
            objective='binary:logistic',
            eval_metric='logloss',
            scale_pos_weight=scale_pos_weight,
            n_estimators=300,
            learning_rate=0.05,
            subsample=0.9,
            colsample_bytree=0.9,
            random_state=42,
            tree_method='hist',
            njobs=1
        )),
    ]
)

""" J’ai encapsulé le prétraitement et le modèle dans un pipeline unique, pour être sûr que la cross-validation et le déploiement utilisent exactement les mêmes transformations."""

In [ ]:
from scipy.stats import loguniform, randint, uniform

pipeline_lr = globals().get('pipeline_lr')
pipeline_gb = globals().get('pipeline_gb')
pipeline_xgb = globals().get('pipeline_xgb')



param_distributions_lr = {
    'classifier__C': loguniform(1e-2, 1e2),
    'classifier__tol': loguniform(1e-6, 1e-3),
}

param_distributions_gb = {
    'classifier__learning_rate': loguniform(1e-3, 1e0),
    'classifier__max_depth': randint(2, 12),
    'classifier__min_samples_leaf': randint(20, 200),
    'classifier__l2_regularization': loguniform(1e-6, 1e0),
    'classifier__max_leaf_nodes': randint(16, 128),
}

param_distributions_xgb = {
    'classifier__n_estimators': randint(200, 600),
    'classifier__max_depth': randint(3, 10),
    'classifier__min_child_weight': randint(1, 10),
    'classifier__gamma': loguniform(1e-3, 1e1),
    'classifier__subsample': uniform(0.6, 0.4),
    'classifier__colsample_bytree': uniform(0.6, 0.4),
    'classifier__reg_lambda': loguniform(1e-3, 1e2),
    'classifier__reg_alpha': loguniform(1e-3, 1e1),
    'classifier__learning_rate': loguniform(1e-3, 5e-1),
}



""" Pour chaque algorithme, j’ai défini un espace d’hyperparamètres réaliste (nombre d’arbres, profondeur, taux d’apprentissage, etc.) que RandomizedSearchCV va explorer, 
avec comme métrique principale l’AUC."""

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

search_spaces = {
    'pipeline_lr': RandomizedSearchCV(
        estimator=pipeline_lr,
        param_distributions=param_distributions_lr,
        n_iter=30,
        scoring='roc_auc',
        cv=5,
        n_jobs=-1,
        random_state=42,
        verbose=1,
    ),
    'pipeline_gb': RandomizedSearchCV(
        estimator=pipeline_gb,
        param_distributions=param_distributions_gb,
        n_iter=40,
        scoring='roc_auc',
        cv=5,
        n_jobs=-1,
        random_state=42,
        verbose=1,
    ),
    'pipeline_xgb': RandomizedSearchCV(
        estimator=pipeline_xgb,
        param_distributions=param_distributions_xgb,
        n_iter=50,
        scoring='roc_auc',
        cv=5,
        n_jobs=-1,
        random_state=42,
        verbose=1,
    ),
}

# Dictionnaire pour stocker les meilleurs résultats et modèles
best_results = dict()

# On boucle sur les modèles et on fit/test chaque RandomizedSearchCV
for name, search in search_spaces.items():
    print(f"Fitting {name} ...")
    search.fit(X_train, y_train)  
    best_results[name] = {
        'best_estimator': search.best_estimator_,
        'best_score': search.best_score_,
        'best_params': search.best_params_,
    }
    print(f"{name} -> Best AUC: {search.best_score_:.4f}")

# Accès facile au meilleur modèle/score/params par nom de pipeline :
# best_results['pipeline_lr']['best_estimator'], etc.

best_results

""" Pour chaque pipeline (prétraitement + modèle), j’ai lancé un RandomizedSearchCV avec une cross-validation stratifiée sur le jeu d’entraînement et un scoring AUC, 
ce qui me permet de comparer les modèles à partir d’une métrique robuste sur données déséquilibrées. """

## <a id="6-evaluation-validation-auc-et-cout-metier"></a>6. Évaluation validation : AUC et coût métier


In [ ]:
from sklearn.metrics import confusion_matrix,  precision_score, recall_score, f1_score, roc_auc_score

# Get predictions for each best estimator on validation set
y_proba_dict = {}
for name, results in best_results.items():
    estimator = results['best_estimator']
    y_proba = estimator.predict_proba(X_valid)[:, 1]  
    y_proba_dict[name] = y_proba
    print(f"{name}: Computed probabilities for {len(y_proba)} validation samples")

def evaluate_with_cost(y_true, y_proba, threshold=0.5, c_fn=10, c_fp=1):
    
    # Predict based on threshold
    y_pred = (y_proba >= threshold).astype(int)
    
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    # Calculate costs
    total_cost = (fn * c_fn) + (fp * c_fp)
    n_samples = len(y_true)
    avg_cost_per_customer = total_cost / n_samples
    
    # Calculate classification metrics
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    roc_auc = roc_auc_score(y_true, y_proba)
    
    results = {
        'confusion_matrix': cm,
        'tn': tn, 'fp': fp, 'fn': fn, 'tp': tp,
        'total_cost': total_cost,
        'avg_cost_per_customer': avg_cost_per_customer,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'roc_auc': roc_auc,
        'threshold': threshold,
        'c_fn': c_fn,
        'c_fp': c_fp,
    }
    
    return results

# Evaluation de chaque modèle avec le seuil par défaut = 0,5
print("=" * 60)
print("Evaluation with threshold=0.5, c_fn=10, c_fp=1")
print("=" * 60)

evaluation_results = {}
for name, y_proba in y_proba_dict.items():
    results = evaluate_with_cost(y_valid, y_proba, threshold=0.5, c_fn=10, c_fp=1)
    evaluation_results[name] = results
    
    print(f"\n{name}:")
    print(f"  Confusion Matrix:\n{results['confusion_matrix']}")
    print(f"  Total Cost: {results['total_cost']:.2f}")
    print(f"  Avg Cost per Customer: {results['avg_cost_per_customer']:.4f}")
    print(f"  Precision: {results['precision']:.4f}")
    print(f"  Recall: {results['recall']:.4f}")
    print(f"  F1-Score: {results['f1_score']:.4f}")
    print(f"  ROC-AUC: {results['roc_auc']:.4f}")

evaluation_results


In [ ]:
# Maintenant qu'on a évalué chaque modèle avec le seuil par défaut, on cherche le seuil optimal pour minimiser le coût métier

# Créer une liste de seuils de 0.01 à 0.99 avec un pas de 0.01
thresholds = [round(t, 2) for t in np.arange(0.01, 1.00, 0.01)]

# On stock le seuil optimal et les métriques associées pour chaque modèle
optimal_results = {}

print("=" * 80)
print("Finding optimal threshold for each model (minimizing cost)")
print("=" * 80)

for name, y_proba in y_proba_dict.items():
    print(f"\n{name}: Testing {len(thresholds)} thresholds...")
    
    # Test all thresholds and find the one with minimum cost
    best_cost = float('inf')
    best_threshold = None
    best_metrics = None
    
    for threshold in thresholds:
        results = evaluate_with_cost(y_valid, y_proba, threshold=threshold, c_fn=10, c_fp=1)
        cost = results['total_cost']
        
        if cost < best_cost:
            best_cost = cost
            best_threshold = threshold
            best_metrics = results
    
    #  AUC sur validation set
    auc_valid = roc_auc_score(y_valid, y_proba)
    
    # Stockage des métriques
    optimal_results[name] = {
        'best_threshold': best_threshold,
        'cost': best_metrics['total_cost'],
        'avg_cost_per_customer': best_metrics['avg_cost_per_customer'],
        'recall_1': best_metrics['recall'],  # Recall for class 1
        'f1_1': best_metrics['f1_score'],   # F1 for class 1
        'precision_1': best_metrics['precision'],
        'auc_valid': auc_valid,
        'confusion_matrix': best_metrics['confusion_matrix'],
        'tp': best_metrics['tp'],
        'fp': best_metrics['fp'],
        'tn': best_metrics['tn'],
        'fn': best_metrics['fn'],
    }
    
    print(f"  Best threshold: {best_threshold:.2f}")
    print(f"  Minimal cost: {best_cost:.2f}")
    print(f"  Avg cost per customer: {best_metrics['avg_cost_per_customer']:.4f}")
    print(f"  Recall (class 1): {best_metrics['recall']:.4f}")
    print(f"  F1 (class 1): {best_metrics['f1_score']:.4f}")
    print(f"  AUC (validation): {auc_valid:.4f}")

print("\n" + "=" * 80)
print("Summary of optimal results:")
print("=" * 80)

# dataframe pour une meilleure visu
summary_data = []
for name, results in optimal_results.items():
    summary_data.append({
        'model': name,
        'best_threshold': results['best_threshold'],
        'cost': results['cost'],
        'recall_1': results['recall_1'],
        'f1_1': results['f1_1'],
        'auc_valid': results['auc_valid'],
    })

summary_df = pd.DataFrame(summary_data)
display(summary_df)

optimal_results
""" Je compare les modèles selon deux axes : une métrique technique (l’AUC sur le jeu de validation) et une métrique métier (coût FN/FP, avec le seuil optimisé). 
Le choix final du modèle tient compte de ces deux dimensions."""


""" Une fois les hyperparamètres optimisés, je n’utilise pas le seuil par défaut de 0.5. J’optimise un seuil spécifique en minimisant un coût métier qui pondère 
plus fortement les faux négatifs (mauvais clients accordés) que les faux positifs (bons clients refusés). Cela permet de rendre le modèle cohérent avec les enjeux économiques : 
un crédit accordé à un mauvais payeur coûte plus cher que la perte de marge associée à un refus injustifié. """

In [ ]:

""" À partir des probabilités de défaut prédites sur le jeu de validation, j’ai optimisé le seuil de décision pour chaque modèle de façon à minimiser un coût métier :
- un faux négatif (mauvais client accepté) coûte 10 unités,
- un faux positif (bon client refusé) coûte 1 unité.

Pour chaque modèle, j’ai balayé les seuils de 0.01 à 0.99, calculé la matrice de confusion et le coût total associé, puis retenu le seuil qui minimise ce coût.
J’obtiens ainsi, pour chaque modèle :
- un seuil optimisé,
- le coût métier minimal atteint,
- des métriques classiques (recall, F1 sur la classe 1, précision)
- et l’AUC sur le jeu de validation.

Ces résultats sont synthétisés dans un tableau comparatif, qui me permet de choisir le modèle final en tenant compte à la fois :
- de la performance globale (AUC),
- et de l’impact économique des erreurs (coût FN/FP)."""

## <a id="7-selection-du-modele-champion-et-interpretation-metier"></a>7. Sélection du modèle champion et interprétation métier


In [ ]:
from sklearn.metrics import accuracy_score

# Étape 1 : choisir le meilleur modèle
# Idée : on veut surtout MINIMISER le coût métier,
# mais on veut aussi un modèle "correct" en AUC et en rappel (recall).

print("=" * 80)
print("Sélection du meilleur modèle (coût, AUC et rappel)")
print("=" * 80)

# On va calculer un "score global" pour chaque modèle.
# Plus ce score est petit plus le cout métier est faible.
best_model_name = None
best_score = float('inf')

for name, results in optimal_results.items():
    # On normalise le coût pour qu'il soit sur une échelle comparable
    # Ici, on divise par 10000 (c'est une approximation) pour éviter d'avoir
    # un coût très grand qui écrase tout le reste.

    normalized_cost = results['cost'] / 10000.0

    # On calcule un score global (composite_score)
    # - On veut : coût petit  -> donc on ajoute le coût (avec un poids)
    # - On veut : AUC grande  -> donc on SOUSTRAIT l'AUC (car on minimise le score)
    # - On veut : recall grand -> donc on SOUSTRAIT le recall
    #
    # Les poids 0.5, 0.3, 0.2 servent juste à donner plus d'importance au coût.

    composite_score = 0.5 * normalized_cost - 0.3 * results['auc_valid'] - 0.2 * results['recall_1']

    """ J’ai exploré les modèles obtenus et retenu celui qui minimise le coût métier tout en gardant une AUC et un rappel satisfaisants. """
    
    print(f"{name}: cost={results['cost']:.2f}, auc={results['auc_valid']:.4f}, recall={results['recall_1']:.4f}, composite={composite_score:.4f}")
    
    if composite_score < best_score:
        best_score = composite_score
        best_model_name = name

print(f"\nModèle sélectionné: {best_model_name}")
print(f"  Seuil (threshold): {optimal_results[best_model_name]['best_threshold']:.2f}")
print(f"  Coût (validation): {optimal_results[best_model_name]['cost']:.2f}")
print(f"  AUC (validation): {optimal_results[best_model_name]['auc_valid']:.4f}")
print(f"  Recall classe 1 (validation): {optimal_results[best_model_name]['recall_1']:.4f}")

# récupérer le meilleur seuil + le meilleur modèle entraîné
best_threshold = optimal_results[best_model_name]['best_threshold']
best_estimator_from_search = best_results[best_model_name]['best_estimator']
best_params = best_results[best_model_name]['best_params']

print(f"\nMeilleurs hyperparamètres: {best_params}")

#  regrouper train + validation pour ré-entraîner un modèle final
# (on utilise plus de données pour apprendre, donc c'est souvent mieux)

print("\n" + "=" * 80)
print("On regroupe TRAIN + VALIDATION")
print("=" * 80)

X_train_full = pd.concat([X_train, X_valid], axis=0).reset_index(drop=True)
y_train_full = pd.concat([y_train, y_valid], axis=0).reset_index(drop=True)

print(f"X_train_full shape: {X_train_full.shape}")
print(f"y_train_full shape: {y_train_full.shape}")

# créer et entraîner le pipeline final
print("\n" + "=" * 80)
print(f"Création du pipeline final pour  {best_model_name} ")
print("=" * 80)


# Ici, best_estimator_from_search est déjà un pipeline complet
# (prétraitement + modèle) avec les bons hyperparamètres
final_pipeline = best_estimator_from_search


print("Entraînement du modèle final sur train_full...")
final_pipeline.fit(X_train_full, y_train_full)
print("Entraînement terminé!")

# prédire sur le jeu de TEST

# On calcule d'abord des probabilités, puis on applique le seuil.

print("\nPrédiction des probabilités sur le jeu de TEST...")
y_proba_test = final_pipeline.predict_proba(X_test)[:, 1]

# Décision finale avec le meilleur seuil
y_pred_test = (y_proba_test >= best_threshold).astype(int)

# évaluation finale sur TEST
# On calcule le coût + quelques métriques classiques.
print("\n" + "=" * 80)
print("Évaluation finale sur le jeu de TEST")
print("=" * 80)

test_results = evaluate_with_cost(y_test, y_proba_test, threshold=best_threshold, c_fn=10, c_fp=1)


auc_test = roc_auc_score(y_test, y_proba_test)
accuracy_test = accuracy_score(y_test, y_pred_test)


final_test_results = {
    'model_name': best_model_name,
    'best_threshold': best_threshold,
    'auc_test': auc_test,
    'cost_test': test_results['total_cost'],
    'avg_cost_per_customer_test': test_results['avg_cost_per_customer'],
    'recall_1_test': test_results['recall'],
    'f1_1_test': test_results['f1_score'],
    'precision_1_test': test_results['precision'],
    'accuracy_test': accuracy_test,
    'confusion_matrix_test': test_results['confusion_matrix'],
    'tp': test_results['tp'],
    'fp': test_results['fp'],
    'tn': test_results['tn'],
    'fn': test_results['fn'],
}

print("\nTest Set Results:")
print(f"  AUC: {auc_test:.4f}")
print(f"  Cost: {test_results['total_cost']:.2f}")
print(f"  Avg Cost per Customer: {test_results['avg_cost_per_customer']:.4f}")
print(f"  Recall (class 1): {test_results['recall']:.4f}")
print(f"  F1 (class 1): {test_results['f1_score']:.4f}")
print(f"  Precision (class 1): {test_results['precision']:.4f}")
print(f"  Accuracy: {accuracy_test:.4f}")
print("\nConfusion Matrix:")
print(test_results['confusion_matrix'])

final_test_results

"""Une fois le modèle champion et son seuil métier choisis sur validation, je le réentraîne sur l’ensemble train+validation avec les meilleurs hyperparamètres, 
puis je l’évalue une seule fois sur le jeu de test tenu à part. Cela me donne une estimation honnête de la performance finale, à la fois en termes d’AUC et de coût métier."""

In [ ]:
""" 
À partir des résultats sur le jeu de validation, j’ai choisi un ‘modèle champion’ en privilégiant d’abord le coût métier (FN beaucoup plus pénalisants que FP), 
puis en tenant compte de l’AUC et du rappel de la classe 1.

Une fois ce modèle et son seuil métier optimisé sélectionnés, je l’ai réentraîné sur l’ensemble du jeu train + validation avec les hyperparamètres trouvés par RandomizedSearchCV.

Enfin, j’ai évalué ce modèle final sur le jeu de test, tenu complètement à part, en calculant :
	•	l’AUC (performance globale de discrimination),
	•	le coût métier total et moyen par client,
	•	le rappel, la F1 et la précision sur la classe 1 (clients défaillants),
	•	l’accuracy et la matrice de confusion.

Ce sont ces résultats sur le test qui servent d’estimation finale et honnête de la performance du système de scoring en conditions réelles.

""" 

In [ ]:
"""
Avec le modèle XGBoost et un seuil métier optimisé à 0,54 :
	•	l’AUC est de 0,76 sur le jeu de test, ce qui indique une bonne capacité de discrimination,
	•	le taux de défaut parmi les crédits acceptés est ramené d’environ 8 % à 4,2 %, soit une division par deux du risque,
	•	le groupe des crédits refusés est fortement enrichi en mauvais payeurs (taux de défaut ≈ 18,5 %),
	•	le coût métier (10×FN + 1×FP) est réduit d’environ 35 % par rapport à une politique naïve qui accorde le crédit à tout le monde.

La précision sur la classe 1 reste faible (≈18,5 %), ce qui est attendu dans un contexte où les défauts sont rares (8 %) et où l’on privilégie la réduction des faux négatifs
 (mauvais clients acceptés) au prix d’un certain nombre de faux positifs (bons clients refusés).

Au regard de la problématique métier – limiter les pertes liées aux défauts de paiement tout en contrôlant l’impact sur les bons clients – 
le modèle répond à l’objectif : il permet de réduire significativement le risque de défaut sur le portefeuille de crédits accordés, tout en minimisant un coût métier explicitement défini.



""" 

In [ ]:
""" 
 Relire les chiffres clés en langage métier

Sur le jeu de test (61 503 clients) avec modèle XGBoost + seuil métier 0,54 :
	•	Matrice de confusion :

\pmatrice
{TN}=42898 & {FP}=13640 
{FN}=1868 & {TP}=3097


Traduction :
	•	TN = 42 898 : bons clients correctement acceptés ✅
	•	FP = 13 640 : bons clients refusés ❌ (manque à gagner)
	•	FN = 1 868 : mauvais payeurs acceptés ❌ (perte en capital)
	•	TP = 3 097 : mauvais payeurs refusés ✅

Rappels :
	•	Positifs (1) = mauvais payeurs = 4 965 (8,1 % de la pop)
	•	Négatifs (0) = bons payeurs = 56 538 (91,9 %)

Metrics principales :
	•	AUC_test = 0,7613
	•	Recall (classe 1, mauvais payeurs) = 0,6238
→ refus : ~62 % des mauvais payeurs.
	•	Precision (classe 1) = 0,1850
→ parmi les clients refusés, ~18,5 % sont vraiment mauvais.
(Donc 81,5 % des refus sont des “bons” qu’on sacrifie)
	•	Accuracy = 0,7478
	•	Coût métier (10×FN + 1×FP) : 32 320
→ Coût moyen par client ≈ 0,5255

⸻

Reformulons la problématique :

“On veut un modèle qui limite les prêts accordés à des gens qui ne remboursent pas (FN), en acceptant de refuser certains bons clients, avec un coût FN = 10 × FP.”

Regardons ça avec des comparaisons simples.

3.1. Si la banque n’utilise aucun modèle

Deux politiques naïves :
	1.	Accorder TOUS les crédits (toujours 0)
	•	FN = tous les mauvais payeurs = 4 965
	•	FP = 0
	•	Coût = 10 × 4 965 = 49 650
	•	Coût moyen ≈ 0,807 par client
	2.	Refuser TOUS les crédits (toujours 1)
	•	FN = 0
	•	FP = tous les bons = 56 538
	•	Coût = 56 538
	•	Coût moyen ≈ 0,919 par client

Notre modèle :
	•	Coût = 32 320
	•	Coût moyen ≈ 0,5255

Conclusion :
Le modèle fait nettement mieux que les deux politiques extrêmes.
Il réduit le coût moyen par client de ~0,81 → ~0,53, donc baisse de ~35 % par rapport à “on accepte tout”.

⸻

Que devient le risque dans le portefeuille accepté ?

Parmi les clients acceptés (prédits 0) :
	•	Acceptés = TN + FN = 42 898 + 1 868 = 44 766
	•	Parmi eux, mauvais payeurs = FN = 1 868

Taux de mauvais payeurs parmi les acceptés :

{Default rate parmi acceptés} = {1868}/{44766} \approx 4,2\%

Alors que dans la population totale :

{Default rate global} = {4965}{61503} \approx 8,1\%

 Notre modèle champion a quasiment divisé par 2 le taux de mauvais payeurs dans le portefeuille de crédits accordés.

Ça, pour un métier risque crédit, c’est très parlant :
	•	“On garde 73 % des demandes acceptées”
	•	mais
	•	“On divise par 2 la proportion de mauvais payeurs dans ce portefeuille”

⸻

Quel est le prix à payer côté manque à gagner (FP) ?

Parmi les bons clients (classe 0, 56 538 au total) :
	•	TN = 42 898 → bons acceptés
	•	FP = 13 640 → bons refusés (manque à gagner)

Taux de bons refusés :

{13\,640}{56\,538} \approx 24\%

Donc :
	•	Environ 76 % des bons clients sont acceptés
	•	Environ 24 % des bons clients sont refusés  (FP)

Vu que :
	•	FN “coûtent 10”
	•	FP “coûtent 1”

Avec ce modèle et ce seuil, on :
	•	accepte ~73 % des demandes de crédit,
	•	tout en réduisant de moitié le taux de mauvais payeurs parmi les clients réellement financés,
	•	pour un coût métier nettement inférieur aux politiques naïves (acceptation ou refus systématique).
Ce compromis est cohérent avec l’hypothèse de départ : un défaut coûte environ 10 fois plus cher qu’un refus injustifié.



	•	Si la banque veut encore moins de mauvais payeurs acceptés (FN), on peut :
	•	augmenter le poids c_fn (>10),
	•	ou choisir un seuil donnant un recall1 plus élevé, même si le coût global remonte légèrement.
	•	Si au contraire elle veut refuser moins de bons clients, on peut :
	•	réduire c_fn/c_fp,
	•	ou choisir un seuil un peu moins conservateur (rappel 1 plus bas, mais moins de FP).


j’ai calibré le modèle sur un ratio de coût FN/FP = 10.
Si le métier juge que ce ratio est trop/peu conservateur, on peut facilement recalibrer le seuil ou les poids pour adapter le modèle à leur tolérance au risque.

⸻


“Le modèle XGBoost sélectionné, avec un seuil optimisé à 0,54, permet :
	•	d’atteindre une AUC de 0,76 (bonne capacité de discrimination),
	•	de refuser ~62 % des clients qui auraient effectivement fait défaut,
	•	de réduire le taux de défaut parmi les crédits accordés d’environ 8,1 % à 4,2 %,
	•	tout en acceptant encore ~73 % des demandes de crédit.

Le coût métier moyen par client (FN pondérés 10 fois plus que les FP) est significativement inférieur aux stratégies naïves (tout accepter ou tout refuser).
On répond donc à la problématique métier en limitant les pertes en capital liées aux défauts, au prix d’un certain nombre de refus sur des clients solvables, 
ce qui est conforme au cadrage initial.


""" 

## <a id="8-suivi-experimental-avec-mlflow"></a>8. Suivi expérimental avec MLflow


# Configuration MLflow

In [ ]:
import mlflow

MLFLOW_DIR.mkdir(parents=True, exist_ok=True)

# Configure MLflow tracking
mlflow.set_tracking_uri(MLFLOW_DIR.as_uri())
mlflow.set_experiment("home_credit_scoring")

experiment_name = "home_credit_scoring"

exp = mlflow.get_experiment_by_name(experiment_name)
if exp is None:
    experiment_id = mlflow.create_experiment(experiment_name)
else:
    experiment_id = exp.experiment_id

print("Experiment ID utilisé :", experiment_id)

## Rassembler les infos

In [ ]:
# Consolidate champion model information
champion_info = {
    'model_name': best_model_name,
    'best_threshold': best_threshold,
    'best_params': best_params,
    # Validation metrics
    'valid_auc': optimal_results[best_model_name]['auc_valid'],
    'valid_cost': optimal_results[best_model_name]['cost'],
    'valid_avg_cost_per_customer': optimal_results[best_model_name]['avg_cost_per_customer'],
    'valid_recall_1': optimal_results[best_model_name]['recall_1'],
    'valid_precision_1': optimal_results[best_model_name]['precision_1'],
    'valid_f1_1': optimal_results[best_model_name]['f1_1'],
    'valid_confusion_matrix': optimal_results[best_model_name]['confusion_matrix'],
    'valid_tp': optimal_results[best_model_name]['tp'],
    'valid_fp': optimal_results[best_model_name]['fp'],
    'valid_tn': optimal_results[best_model_name]['tn'],
    'valid_fn': optimal_results[best_model_name]['fn'],
    # Test metrics
    'test_auc': final_test_results['auc_test'],
    'test_cost': final_test_results['cost_test'],
    'test_avg_cost_per_customer': final_test_results['avg_cost_per_customer_test'],
    'test_recall_1': final_test_results['recall_1_test'],
    'test_precision_1': final_test_results['precision_1_test'],
    'test_f1_1': final_test_results['f1_1_test'],
    'test_accuracy': final_test_results['accuracy_test'],
    'test_confusion_matrix': final_test_results['confusion_matrix_test'],
    'test_tp': final_test_results['tp'],
    'test_fp': final_test_results['fp'],
    'test_tn': final_test_results['tn'],
    'test_fn': final_test_results['fn'],
}

print("Champion model summary:")
for key, value in champion_info.items():
    if isinstance(value, (float, int)):
        print(f"  {key}: {value}")
    else:
        print(f"  {key}: {value}")

champion_info

## Run MLflow pour le modèle champion 

In [ ]:
import shap
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
import mlflow.sklearn

#  Variables importantes à passer à MLflow : 
# - best_model_name
# - best_threshold
# - best_params
# - champion_info
# - final_pipeline           (pipeline champion entraîné sur X_train_full)
# - y_test, y_proba_test
# - y_valid, thresholds      (liste des seuils explorés)
# - evaluate_with_cost
# - y_proba_dict             (probas sur X_valid pour chaque modèle)
# - scale_pos_weight         (pour XGBoost)

mlflow_run_name = f"champion_{best_model_name}"
print(f"Logging MLflow run: {mlflow_run_name}")

FIGURES_DIR = Path("/Users/ely/Developer/home_credit_project01/home_credit_project/artifacts/figures")


# Pour la courbe coût vs seuil du champion sur validation
y_proba_valid_champion = y_proba_dict[best_model_name]

with mlflow.start_run(
    run_name=mlflow_run_name,
    experiment_id=experiment_id,):
    # ───────────────────────────── Params ─────────────────────────────
    param_payload = {
        "model_name": best_model_name,
        "best_threshold": best_threshold,
        "c_fn": 10,
        "c_fp": 1,
        "scale_pos_weight": scale_pos_weight,
        "n_train_full": len(X_train_full),
        "n_valid": len(y_valid),
        "n_test": len(y_test),
        # hyperparamètres XGBoost (castés en str pour être sûrs)
        **{str(k): str(v) for k, v in best_params.items()},
    }
    mlflow.log_params(param_payload)

    # ───────────────────────────── Metrics ─────────────────────────────
    # Validation
    validation_metrics = {
        "valid_auc": champion_info["valid_auc"],
        "valid_cost": champion_info["valid_cost"],
        "valid_avg_cost_per_customer": champion_info["valid_avg_cost_per_customer"],
        "valid_recall_1": champion_info["valid_recall_1"],
        "valid_precision_1": champion_info["valid_precision_1"],
        "valid_f1_1": champion_info["valid_f1_1"],
    }

    # Test
    test_metrics = {
        "test_auc": champion_info["test_auc"],
        "test_cost": champion_info["test_cost"],
        "test_avg_cost_per_customer": champion_info["test_avg_cost_per_customer"],
        "test_recall_1": champion_info["test_recall_1"],
        "test_precision_1": champion_info["test_precision_1"],
        "test_f1_1": champion_info["test_f1_1"],
        "test_accuracy": champion_info["test_accuracy"],
    }

    mlflow.log_metrics({**validation_metrics, **test_metrics})

    # ───────────── Confusion matrices (JSON artefact) ─────────────
    mlflow.log_dict(
        {
            "validation_confusion_matrix": champion_info[
                "valid_confusion_matrix"
            ].tolist(),
            "test_confusion_matrix": champion_info["test_confusion_matrix"].tolist(),
            "validation_counts": {
                "tp": champion_info["valid_tp"],
                "fp": champion_info["valid_fp"],
                "tn": champion_info["valid_tn"],
                "fn": champion_info["valid_fn"],
            },
            "test_counts": {
                "tp": champion_info["test_tp"],
                "fp": champion_info["test_fp"],
                "tn": champion_info["test_tn"],
                "fn": champion_info["test_fn"],
            },
        },
        artifact_file="confusion_matrices.json",
    )

    # ───────────── ROC curve (TEST) ─────────────
    fpr, tpr, _ = roc_curve(y_test, y_proba_test)
    plt.figure()
    plt.plot(fpr, tpr, label=f"ROC test (AUC = {champion_info['test_auc']:.3f})")
    plt.plot([0, 1], [0, 1], linestyle="--", label="Random")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve - Test set")
    plt.legend(loc="lower right")
    plt.tight_layout()
    roc_path = FIGURES_DIR / "roc_curve_test.png"
    plt.savefig(roc_path, bbox_inches="tight")
    plt.close()
    mlflow.log_artifact(str(roc_path))

    # ───────────── Coût métier vs seuil (VALIDATION) ─────────────
    costs = []
    for t in thresholds:
        res_t = evaluate_with_cost(y_valid, y_proba_valid_champion, threshold=t, c_fn=10, c_fp=1)
        costs.append(res_t["total_cost"])

    plt.figure()
    plt.plot(thresholds, costs, marker=".")
    plt.axvline(best_threshold, color="red", linestyle="--", label=f"Best threshold = {best_threshold:.2f}")
    plt.xlabel("Threshold")
    plt.ylabel("Total cost (10*FN + 1*FP)")
    plt.title("Business cost vs threshold (validation)")
    plt.legend()
    plt.tight_layout()
    cost_curve_path = FIGURES_DIR /  "cost_vs_threshold_valid.png"
    plt.savefig(cost_curve_path, bbox_inches="tight")
    plt.close()
    mlflow.log_artifact(str(cost_curve_path))

    # ───────────── Confusion matrix (TEST) en heatmap ─────────────
    cm_test = champion_info["test_confusion_matrix"]
    plt.figure()
    plt.imshow(cm_test, interpolation="nearest")
    plt.title("Confusion Matrix - Test set")
    plt.colorbar()
    tick_marks = [0, 1]
    plt.xticks(tick_marks, ["Pred 0", "Pred 1"])
    plt.yticks(tick_marks, ["True 0", "True 1"])

    # annotations
    for i in range(cm_test.shape[0]):
        for j in range(cm_test.shape[1]):
            plt.text(
                j,
                i,
                int(cm_test[i, j]),
                ha="center",
                va="center",
                color="white" if cm_test[i, j] > cm_test.max() / 2 else "black",
            )

    plt.ylabel("True label")
    plt.xlabel("Predicted label")
    plt.tight_layout()
    cm_path = FIGURES_DIR /  "confusion_matrix_test.png"
    plt.savefig(cm_path, bbox_inches="tight")
    plt.close()
    mlflow.log_artifact(str(cm_path))

    # ───────────────────────────── SHAP Values ─────────────────────────────
    
    # Extraire le modèle et le preprocessor de la pipeline
    model = final_pipeline.named_steps['classifier']
    preprocessor_steps = final_pipeline[:-1]
    
    # Transformer les données test (on utilise un échantillon pour la rapidité)
    X_test_sample = X_test.sample(n=min(1000, len(X_test)), random_state=42)
    X_test_transformed = preprocessor_steps.transform(X_test_sample)
    feature_names = preprocessor_steps.get_feature_names_out()
    
    # Calcul SHAP
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_test_transformed)
    
    # Pour XGBoost binaire, shap_values est directement un array (pas une liste)
    # Mais on vérifie au cas où
    if isinstance(shap_values, list):
        shap_values = shap_values[1]
    
    # 1. Summary plot (beeswarm)
    plt.figure(figsize=(12, 8))
    shap.summary_plot(shap_values, X_test_transformed, feature_names=feature_names, max_display=20, show=False)
    plt.tight_layout()
    shap_summary_path = FIGURES_DIR /  "shap_summary_plot.png"
    plt.savefig(shap_summary_path, bbox_inches="tight", dpi=150)
    plt.show()  # Affiche dans le notebook
    plt.close()
    mlflow.log_artifact(str(shap_summary_path))
    
    # 2. Bar plot (importance moyenne absolue)
    plt.figure(figsize=(10, 8))
    shap.summary_plot(shap_values, X_test_transformed, feature_names=feature_names, plot_type="bar", max_display=20, show=False)
    plt.tight_layout()
    shap_bar_path = FIGURES_DIR /  "shap_feature_importance.png"
    plt.savefig(shap_bar_path, bbox_inches="tight", dpi=150)
    plt.show()
    plt.close()
    mlflow.log_artifact(str(shap_bar_path))
    
    print("SHAP plots logged to MLflow.")


    # ───────────── Enregistrement du modèle champion ─────────────
    # Pipeline complet (préprocesseur + XGBoost) dans le Model Registry
    mlflow.sklearn.log_model(
        sk_model=final_pipeline,
        artifact_path="model",
        registered_model_name="home_credit_xgb_champion",
    )

print("MLflow logging for champion model completed.")
""" 

Je crée un run MLflow dédié au modèle champion et j’y loggue :
	•	les hyperparamètres issus du RandomizedSearchCV,
	•	le seuil métier optimisé,
	•	les métriques sur le jeu de validation,
	•	et les métriques finales sur le jeu de test.
également les features importances via SHAP (summary plot et bar plot).

"""

In [ ]:
import json
import joblib
import datetime as dt


cwd = Path.cwd()
PROJECT_ROOT = cwd.parent if cwd.name == "notebooks" else cwd

MODELS_DIR = PROJECT_ROOT / "artifacts" / "models"
MODELS_DIR.mkdir(parents=True, exist_ok=True)

PIPELINE_PATH = MODELS_DIR / "champion_pipeline.joblib"
METADATA_PATH = MODELS_DIR / "champion_metadata.json"

assert "final_pipeline" in globals(), "final_pipeline introuvable."
assert "best_threshold" in globals(), "best_threshold introuvable."

# --------- 3) Sauvegarder le pipeline sklearn (préprocess + modèle) ---------
joblib.dump(final_pipeline, PIPELINE_PATH)

# --------- 4) Sauvegarder les métadonnées (seuil + infos utiles) ---------
def _to_jsonable(x):
    # convertit types numpy/pandas en types JSON-friendly
    try:
        import numpy as np
        if isinstance(x, (np.integer,)):
            return int(x)
        if isinstance(x, (np.floating,)):
            return float(x)
        if isinstance(x, (np.ndarray,)):
            return x.tolist()
    except Exception:
        pass
    return x

metadata = {
    "exported_at": dt.datetime.now().isoformat(),
    "best_threshold": float(best_threshold),
    "c_fn": 10.0,
    "c_fp": 1.0,
    "model_name": globals().get("best_model_name", "unknown"),
    "best_params": {str(k): _to_jsonable(v) for k, v in globals().get("best_params", {}).items()},
}

with METADATA_PATH.open("w", encoding="utf-8") as f:
    json.dump(metadata, f, indent=2, ensure_ascii=False)

# --------- 5) Afficher un résumé ---------
print("✅ Champion exporté !")
print("Pipeline :", PIPELINE_PATH)
print("Metadata :", METADATA_PATH)
print(f"Pipeline size: {PIPELINE_PATH.stat().st_size/1024/1024:.2f} MB")
print("Threshold :", metadata["best_threshold"])